## Загрузка библиотек

- unsloth
- transformers
- trl
- peft
- bitsandbytes

In [ ]:
%%capture

# Install Unsloth, Xformers (Flash Attention) and more!

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install datasets

## Импорт зависимостей

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments
import torch

## Константы

In [ ]:
max_seq_length = 1024
dtype = torch.bfloat16
load_in_4bit = False 
load_in_8bit = True 
full_finetuning = True
test_size = 0.1

model_name = "Qwen/Qwen3-1.7B"

cache_dir = "/mode_cache/"
save_dir = f"/outputs/{model_name}/tuned"
output_dir=f"/outputs/{model_name}/outputs"

## Загрузака модели и dataset

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    load_in_8bit = load_in_8bit,
    full_finetuning = full_finetuning
)

# Опционально снизить VRAM
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

RAW_IFT_LINK = "https://raw.githubusercontent.com/nschpy/shinkai-dataset/refs/heads/main/ift.dataset.json"

ds = load_dataset("json", data_files=RAW_IFT_LINK, cache_dir=cache_dir)
ds = ds.train_test_split(test_size=test_size, shuffle=True, seed=42)

## Подготовка данных


In [ ]:
def formatting_func(batch):
    return {
        "text": [
            f"{input_text}\n\n{output_text}"
            for input_text, output_text in zip(batch["input"], batch["output"])
        ]
    }

# ✅ 2. Применение через map(..., batched=True)
train_dataset = ds["train"].map(formatting_func, batched=True)

## Подготовка к SFT

In [ ]:
# ✅ 5. SFTConfig — указываем готовое поле text
sft_config = SFTConfig(
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False,
    learning_rate = 5e-5,
    eos_token=tokenizer.eos_token,
    pad_token=tokenizer.pad_token,
    output_dir=""
    completion_only_loss = True,
    logging_steps=10,
    save_strategy="epoch",
    output_dir=save_dir,
    optim="adamw_torch_fused",
    seed= 42,
    report_to = []
)

trainer = SFTTrainer(
    model = model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    args = sft_config,
)

## Обучение

In [ ]:
trainer.train()
trainer.save_model(output_dir)

## Скачать модель

In [ ]:
!zip -r model1.zip outputs/full_sft1/merged

from google.colab import files

files.download('model1.zip')